In [12]:
import pandas as pd
import numpy as np

#local 読み込み
dataPath = './datasets/'
titlePath = './data/titledata/'
keyPath = './data/keyworddata/'
storyPath = './data/storydata/'
allPath =  './data/all/'

In [ ]:
from janome.analyzer import Analyzer
from janome.charfilter import UnicodeNormalizeCharFilter, RegexReplaceCharFilter
from janome.tokenizer import Tokenizer as JanomeTokenizer  # sumyのTokenizerと名前が被るため
from janome.tokenfilter import POSKeepFilter, ExtractAttributeFilter

text = """転職 Advent Calendar 2016 - Qiitaの14日目となります。 少しポエムも含みます。
今年11月にSIerからWebサービスの会社へ転職しました。
早くから退職することを報告していたこともあって、幸いにも有給消化として１ヶ月のお休みをいただくことができました（これでも10日ほど余らせてしまいました）。
# ・・・ (省略) ・・・
だからこそ、有給消化期間はなんとしてでももぎ取るようにしましょう。"""

# 1行1文となっているため、改行コードで分離
sentences = [t for t in text.split('\n')]
for i in range(2):
    print(sentences[i])
# 転職 Advent Calendar 2016 - Qiitaの14日目となります。 少しポエムも含みます。
# 今年11月にSIerからWebサービスの会社へ転職しました。

# 形態素解析器を作る
analyzer = Analyzer(
    [UnicodeNormalizeCharFilter(), RegexReplaceCharFilter(r'[(\)「」、。]', ' ')],  # ()「」、。は全てスペースに置き換える
    JanomeTokenizer(),
    [POSKeepFilter(['名詞', '形容詞', '副詞', '動詞']), ExtractAttributeFilter('base_form')]  # 名詞・形容詞・副詞・動詞の原型のみ
)

# 抽出された単語をスペースで連結
# 末尾の'。'は、この後使うtinysegmenterで文として分離させるため。
corpus = [' '.join(analyzer.analyze(s)) + '。' for s in sentences]
for i in range(2):
    print(corpus[i])
# 転職 Advent Calendar 2016 - Qiita 14 日 目 なる 少し ポエム 含む。
# 今年 11 月 SIer Web サービス 会社 転職 する。

In [ ]:
import MeCab

# MeCab による単語への分割関数,(名詞,形容詞,動詞)のみ残す
def MorphologicalAnalysis(sentences):
  if sentences is np.nan:#欠損値は欠損値のまま返す
    return np.NaN
  result = []
  text_list = sentences.split("。")
  for text in text_list:
    tagger = MeCab.Tagger()
    words = []
    for c in tagger.parse(text).splitlines()[:-1]:
      surface, feature = c.split('\t')
      pos = feature.split(',')[0]
      if pos in ['名詞', '動詞', '形容詞', '副詞']:
        words.append(surface)
    result.append(" ".join(words)+"。")
  return ' '.join(result)

#形態素解析結果を'mecab'に代入
data = pd.read_csv(storyPath+'cleaning.csv', index_col=0)
data['summary_mecab'] = data['cleaning'].apply(MorphologicalAnalysis)
data.to_csv(storyPath+'summary_mecab.csv')

In [32]:
from tqdm import tqdm
#セットアップ
tqdm.pandas()

from sudachipy import tokenizer
from sudachipy import dictionary

tokenizer_obj = dictionary.Dictionary().create()

# sudachi による単語への分割関数,(名詞,形容詞,動詞)のみ残す
def AnalysisBySudachi(sentences):
  if sentences is np.nan:#欠損値は欠損値のまま返す
    return np.NaN
  text_list = sentences.split("。")
  results_list = []
  for text in text_list:
    mode = tokenizer.Tokenizer.SplitMode.C #モードCの一番長い形で分ける
    results =[m.surface() for m in tokenizer_obj.tokenize(text, mode)]
    word_list = []
    for word in results:
      if (word != "") and (word !="、"): #何故か分かち書きの結果として空白データ（''）ができたための省く処理
        normalize = tokenizer_obj.tokenize(word, mode)[0].normalized_form() #正規化してなるべく言葉の揺れを無くす　e.g. 打込む → 打ち込む かつ丼 → カツ丼
        pos = tokenizer_obj.tokenize(normalize, mode)[0].part_of_speech()[0]
        if pos in  ["名詞", "動詞", "形容詞", "副詞"]:  # 対象とする品詞を指定
          word = tokenizer_obj.tokenize(normalize, mode)[0].dictionary_form()
          word_list.append(word)
    results_list.append(" ".join(word_list) + "。") #スペースで繋げていく
  return " ".join(results_list)

#形態素解析結果を'sudachi'に代入
data = pd.read_csv(storyPath+'cleaning.csv', index_col=0)
data['summay_origin'] = data.cleaning.apply(AnalysisBySudachi)
data.to_csv(storyPath+'summay_origin.csv')

ImportError: cannot import name 'Label' from 'pandas._typing' (C:\Users\ngnls\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\_typing.py)

In [6]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

corpus = [
    '転職 Advent Calendar 2016 - Qiita 14 日 目 なる 少し ポエム 含む。',
    '今年 11 月 SIer Web サービス 会社 転職 する。'
]

# 連結したcorpusを再度tinysegmenterでトークナイズさせる
parser = PlaintextParser.from_string(''.join(corpus), Tokenizer('japanese'))

# LexRankで要約を2文抽出
summarizer = LexRankSummarizer()
summarizer.stop_words = [' ']  # スペースも1単語として認識されるため、ストップワードにすることで除外する

summary = summarizer(document=parser.document, sentences_count=2)

# 元の文を表示
for sentence in summary:
    print(sentences[corpus.index(sentence.__str__())])

転職 Advent Calendar 2016 - Qiitaの14日目となります。 少しポエムも含みます。
今年11月にSIerからWebサービスの会社へ転職しました。
